# God Pipeline

In [ ]:
import sys


import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#1. load environment variables and data

# load environment variables
load_dotenv()

#add working directory to sys path to execute utils/dataset.py
working_dir = os.environ.get("WORKING_DIRECTORY")
sys.path.insert(0, working_dir)

from utils.dataset import get_data 

from utils.pipeline_moduls import fs_colinearity, fs_vif, outlier_label, outlier_num, dim_reduction

df = get_data()

df.head(10)


In [ ]:
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=[np.number]).columns
#drop 'quality' from numerical features (its a series)
numerical_features = numerical_features.drop('quality')
label = pd.Series('quality')

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

## Cleaning Pipeline

### Modules

#### Label Outlier Detection

In [ ]:
outlier_detection_label = FunctionTransformer(outlier_label).set_output(transform="pandas")


#### Feature Outlier Detection

In [ ]:
outlier_detection = FunctionTransformer(outlier_num).set_output(transform="pandas")

#### Feature Selection

In [ ]:
import json

def feature_selection(df,colinearity_threshold=0.5, correlation_threshold=0.08, vif_threshold=5):
    dropped_features = []
    dropped_features_set = set(dropped_features)

    # Add elements from fs_colinearity to dropped_features_set
    dropped_features_set.update(fs_colinearity(df, colinearity_threshold, correlation_threshold))

    # Add elements from fs_vif to dropped_features_set
    dropped_features_set.update(fs_vif(df, correlation_threshold, vif_threshold))

    # Convert dropped_features_set back to a list
    dropped_features = list(dropped_features_set)
    print("Dropping Features: ", dropped_features)
    # Drop the features in dropped_features from the DataFrame
    df = df.drop(columns=dropped_features)

    # Save dropped features list to a JSON file
    with open('dropped_features.json', 'w') as f:
        json.dump(dropped_features, f)
    return df
feature_selection = FunctionTransformer(feature_selection).set_output(transform="pandas")

### Pipeline Building

In [ ]:
cleaning_pipeline = Pipeline(steps=[
])

#### Sub-Pipeline: Categorical Features

In [ ]:
categorical_imputer = SimpleImputer(strategy="most_frequent").set_output(transform="pandas")

#pipeline for categorical features
categorical_pipeline = Pipeline(steps=[])
categorical_pipeline.steps.append(('imputer', categorical_imputer))
categorical_pipeline.steps.append(('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")))

#### Sub-Pipeline: Numerical Features

In [ ]:
#pipeline for numerical features
numeric_pipeline = Pipeline(steps=[])
numerical_imputer = SimpleImputer(strategy="mean").set_output(transform="pandas")

numeric_pipeline.steps.append(('imputer', numerical_imputer))
numeric_pipeline.steps.append(('outlier_detection', outlier_detection))

#### Sub-Pipeline: Label

In [ ]:
#pipeline for label
label_pipeline = Pipeline(steps=[])
label_pipeline.steps.append(('imputer', numerical_imputer))


### Pipeline

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('label', label_pipeline, label)
    ]).set_output(transform="pandas")
cleaning_pipeline.steps.append(('preprocessor', preprocessor))
cleaning_pipeline.steps.append(("outlier_detection_label", outlier_detection_label))
cleaning_pipeline.steps.append(('feature_selection', feature_selection))
cleaning_pipeline

## Training Pipeline

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

models = [
    {
        "name": "LinearRegression",
        "estimator": LinearRegression(),
        "hyperparameters":
            {
                "fit_intercept": [True, False],
                "copy_X": [True, False],
                "n_jobs": [-1]
            }
    },
    {
        "name": "DecisionTreeRegressor",
        "estimator": DecisionTreeRegressor(),
        "hyperparameters":
            {
                "criterion": ["squared_error"],
                "splitter": ["best", "random"],
                "max_depth": [None, 2, 5, 10],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 5, 10]
            }
    },
    {
        "name": "RandomForestRegressor",
        "estimator": RandomForestRegressor(),
        "hyperparameters":
            {
                "n_estimators": [100, 200],
                "criterion": ["squared_error"],
                "max_depth": [None, 2, 5, 10],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 5, 10],
                "n_jobs": [-1]
            }
    },
    {
        "name": "Gradient Boosting Regressor",
        "estimator": GradientBoostingRegressor(),
        "hyperparameters":
        {
                "n_estimators": [100, 200, 500],
                "max_depth": [None, 2, 5, 10],
                "min_samples_split": [2, 5, 10],
                "learning_rate": [0.01, 0.011, 0.012],
                "loss": ["squared_error"],
        }
    }
]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pickle


best_models_runtime = []

def god_function(dirty_df):
    for model in models:
        print(model["name"])
        print("-"*len(model["name"]))
        clean_df = pd.DataFrame(cleaning_pipeline.fit_transform(dirty_df))
        X = clean_df.drop('label__quality', axis=1)
        y = clean_df['label__quality']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
        grid = GridSearchCV(model["estimator"], model["hyperparameters"], cv=5, n_jobs=-1)
        grid = grid.fit(X_train, y_train)
        print("Best Parameters:")
        print(grid.best_params_)
        print("")
        print("Best Score:", grid.best_score_, "\t", "Test Score:", grid.score(X_test, y_test))
        print("Fit Time:", grid.refit_time_)
        print("")
        #save best estimator from grid with pickle
        with open("./models/" + model["name"] + '.pkl', 'wb') as f:
            pickle.dump(grid.best_estimator_, f)
        
        #save best model runtime
        best_models_runtime.append({
            "name": model["name"], 
            "runtime": grid.refit_time_,
            "best_score": grid.best_score_,
            "best_params": grid.best_params_
        })

god_function(df)

# TESTING

In [ ]:
def calculate_significant_features(X, y, model):
    coefficients = model.coef_
    intercept = model.intercept_


    residuals = y - model.predict(X)

    n = len(y)
    p = X.shape[1]
    df = n - p - 1

    mse = np.sum(residuals ** 2) / df
    variance_covariance_matrix = mse * np.linalg.inv(np.dot(X.T, X))
    standard_errors = np.sqrt(np.diagonal(variance_covariance_matrix))


    t_values = coefficients / standard_errors
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df))

    headers = ['Feature', 'Coefficient', 'Standard Error', 't-value', 'p-value']

    prediction_metrics = pd.DataFrame(columns=headers)
    for i in range(len(coefficients)):
        prediction_metrics.loc[i] = [X.columns.values[i], coefficients[i], standard_errors[i], t_values[i], p_values[i]]

    #remove rows with p-value > 0.05
    features_to_remove = prediction_metrics[prediction_metrics['p-value'] > 0.05]['Feature'].values
    print("Removing features: ", features_to_remove)
    prediction_metrics = prediction_metrics[prediction_metrics['p-value'] < 0.05]
    return prediction_metrics

In [ ]:
test_models = [{
        "name": "LinearRegression",
        "estimator": LinearRegression(),
        "hyperparameters":
            {
                "fit_intercept": [True, False],
                "copy_X": [True, False],
                "n_jobs": [-1]
            }
    },
    {
        "name": "Support Vector Machine",
        "estimator": SVR(),
        "hyperparameters":
        {
            "kernel": ["linear", "poly", "rbf", "sigmoid"],
            "degree": [1, 2, 3, 4, 5],
            "gamma": ["scale", "auto"],
            "C": [0.1, 1, 10, 100, 1000],
            "epsilon": [0.1, 0.2, 0.3, 0.4, 0.5]
    }
    }
    ]
dirty_df = df.copy(deep=True)
for model in test_models:
    print(model["name"])
    print("-"*len(model["name"]))
    pipeline = cleaning_pipeline
    #pipeline.steps.pop(2)
    clean_df = pd.DataFrame(pipeline.fit_transform(dirty_df))
    X = clean_df.drop('label__quality', axis=1)
    y = clean_df['label__quality']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    grid = GridSearchCV(model["estimator"], model["hyperparameters"], cv=5, n_jobs=-1)
    grid = grid.fit(X_train, y_train)
    print("Best Parameters:")
    print(grid.best_params_)
    print("")
    print("Best Score:", grid.best_score_, "\t", "Test Score:", grid.score(X_test, y_test))
    print("Fit Time:", grid.refit_time_)
    print("")
    best_model = grid.best_estimator_
    significant_features = calculate_significant_features(X_train, y_train, best_model)
    #keep columns of X only if they are present in significant_features
    X_train = X_train[significant_features['Feature'].values]
    X_test = X_test[significant_features['Feature'].values]
    grid = GridSearchCV(model["estimator"], model["hyperparameters"], cv=5, n_jobs=-1)
    grid = grid.fit(X_train, y_train)
    print("Best Parameters:")
    print(grid.best_params_)
    print("")
    print("Best Score:", grid.best_score_, "\t", "Test Score:", grid.score(X_test, y_test))
    print("Fit Time:", grid.refit_time_)
    print("")


In [ ]:
best_model.score()

# END TESTING

# Simulation Meth-Daten

In [ ]:
#load best model with pickle
best_model = pickle.load(open('best_model__random_forest.pkl','rb'))
#select randomly 1000 data points from df and drop selected ones
df_validation = df.sample(n=1000, random_state=1)
clean_df = df.drop(df_validation.index)

X = clean_df.drop('label__quality', axis=1)
y = clean_df['label__quality']

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=200, stratify=y)

print(best_model.score(X_validation, y_validation))

In [ ]:
df_validation = df.sample(n=1000, random_state=42)

clean_df = df.drop(df_validation.index)
clean_df = pd.DataFrame(cleaning_pipeline.fit_transform(clean_df))

X_clean = clean_df.drop('label__quality', axis=1)
y_clean = clean_df['label__quality']

X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_clean, y_clean, test_size=0.2, random_state=200, stratify=y_clean)
best_model = RandomForestRegressor(criterion = 'squared_error', max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200)
best_model = best_model.fit(X_train_clean, y_train_clean)


In [ ]:
best_model.score(X_test_clean, y_test_clean)

In [ ]:
df_validation = pd.DataFrame(cleaning_pipeline.fit_transform(df_validation))
X_validation = df_validation.drop('label__quality', axis=1)
y_validation = df_validation['label__quality']

best_model.score(X_validation, y_validation)